# Tilt and Compliance Corrections for OBS Data
This notebook contains the functions used to conduct tilt and compliance corrections for OBS records. Examples using data directly downloaded from IRIS are also provided at the end of this notebook.

In [ ]:
#import needed packages.
from collections import OrderedDict
from utilities import sta_info_from_inv,plot_trace,segment_interpolate
# from utilities import qml_to_event_list
from TCcorrection_funcs import docorrection,gettransfer, maxcompfreq
import sys
import time
import scipy
import obspy
import pyasdf
import datetime
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from IPython.display import clear_output
from obspy.core.event import Catalog
from obspy.geodetics.base import gps2dist_azimuth
try:
    from obspy.core.event import readEvents
except Exception as e:
    from obspy import read_events as readEvents

In [ ]:
# ##################### qml_to_event_list #####################################
# modified from obspyDMT.utils.event_handler.py
def qml_to_event_list(events_QML):
    """
    convert QML to event list
    :param events_QML:
    :return:
    """
    events = []
    for i in range(len(events_QML)):
        try:
            event_time = events_QML.events[i].preferred_origin().time or \
                         events_QML.events[i].origins[0].time
            event_time_month = '%02i' % int(event_time.month)
            event_time_day = '%02i' % int(event_time.day)
            event_time_hour = '%02i' % int(event_time.hour)
            event_time_minute = '%02i' % int(event_time.minute)
            event_time_second = '%02i' % int(event_time.second)

            if not hasattr(events_QML.events[i], 'preferred_mag'):
                events_QML.events[i].preferred_mag = \
                    events_QML.events[i].magnitudes[0].mag
                events_QML.events[i].preferred_mag_type = \
                    events_QML.events[i].magnitudes[0].magnitude_type
                events_QML.events[i].preferred_author = 'None'
            else:
                if not hasattr(events_QML.events[i], 'preferred_author'):
                    if events_QML.events[i].preferred_magnitude().creation_info:
                        events_QML.events[i].preferred_author = \
                            events_QML.events[i].preferred_magnitude().creation_info.author
                    elif events_QML.events[i].magnitudes[0].creation_info:
                        events_QML.events[i].preferred_author = \
                            events_QML.events[i].magnitudes[0].creation_info.author
        except Exception as error:
            print(error)
            continue
        try:
            if not events_QML.events[i].focal_mechanisms == []:
                if events_QML.events[i].preferred_focal_mechanism()['moment_tensor']['tensor']:
                    focal_mechanism = [
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_rr'],
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_tt'],
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_pp'],
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_rt'],
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_rp'],
                        events_QML.events[i].preferred_focal_mechanism()
                        ['moment_tensor']['tensor']['m_tp']]
                else:
                    found_foc_mech = False
                    for foc_mech_qml in events_QML.events[i].focal_mechanisms:
                        if foc_mech_qml['moment_tensor']['tensor']:
                            focal_mechanism = [
                                foc_mech_qml['moment_tensor']['tensor']['m_rr'],
                                foc_mech_qml['moment_tensor']['tensor']['m_tt'],
                                foc_mech_qml['moment_tensor']['tensor']['m_pp'],
                                foc_mech_qml['moment_tensor']['tensor']['m_rt'],
                                foc_mech_qml['moment_tensor']['tensor']['m_rp'],
                                foc_mech_qml['moment_tensor']['tensor']['m_tp']
                            ]
                            found_foc_mech = True
                            break
                    if not found_foc_mech:
                        focal_mechanism = False
            else:
                focal_mechanism = False
        except AttributeError:
            print("[WARNING] focal_mechanism does not exist for " \
                  "event: %s -- set to False" % (i+1))
            focal_mechanism = False
        except TypeError:
            focal_mechanism = False
        except Exception as error:
            print(error)
            focal_mechanism = False

        try:
            if not events_QML.events[i].focal_mechanisms == []:
                source_duration = [
                    events_QML.events[i].preferred_focal_mechanism()
                    ['moment_tensor']['source_time_function']['type'],
                    events_QML.events[i].preferred_focal_mechanism()
                    ['moment_tensor']['source_time_function']
                    ['duration']]
                if not source_duration[1]:
                    source_duration = mag_duration(
                        mag=events_QML.events[i].preferred_mag)
            else:
                source_duration = mag_duration(
                    mag=events_QML.events[i].preferred_mag)
        except AttributeError:
            print("[WARNING] source duration does not exist for " \
                  "event: %s -- set to False" % (i+1))
            source_duration = False
        except TypeError:
            source_duration = False
        except Exception as error:
            print(error)
            source_duration = False

        try:
            events.append(OrderedDict(
                [('number', i+1),
                 ('latitude',
                  events_QML.events[i].preferred_origin().latitude or
                  events_QML.events[i].origins[0].latitude),
                 ('longitude',
                  events_QML.events[i].preferred_origin().longitude or
                  events_QML.events[i].origins[0].longitude),
                 ('depth',
                  events_QML.events[i].preferred_origin().depth/1000. or
                  events_QML.events[i].origins[0].depth/1000.),
                 ('datetime', event_time),
                 ('magnitude',
                  events_QML.events[i].preferred_mag),
                 ('magnitude_type',
                  events_QML.events[i].preferred_mag_type),
                 ('author',
                  events_QML.events[i].preferred_author),
                 ('event_id', str(event_time.year) +
                  event_time_month + event_time_day + '_' +
                  event_time_hour + event_time_minute +
                  event_time_second + '.a'),
                 ('origin_id', events_QML.events[i].preferred_origin_id or
                  events_QML.events[i].origins[0].resource_id.resource_id),
                 ('focal_mechanism', focal_mechanism),
                 ('source_duration', source_duration),
                 ('flynn_region', 'NAN'),
                 ]))
        except Exception as error:
            print(error)
            continue
    return events

In [ ]:
evt_start= '2012-01-01'
evt_end = '2012-01-31'
evlon_min = -133
evlon_max = -116
evlat_min = 38
evlat_max = 52
evmag_min = 5.0
evmag_max = 10.0
evdep_min = 0
evdep_max = 100

cat=Catalog()
cat = readEvents('http://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/COMBO/combo.ndk')
# cat=readEvents('./gcmtcombo.ndk')
filt1 = 'time >= %s' % evt_start
filt2 = 'time <= %s' % evt_end
cat = cat.filter(filt1, filt2)

filt1 = 'magnitude >= %s' % evmag_min
filt2 = 'magnitude <= %s' % evmag_max
cat = cat.filter(filt1, filt2)

filt1 = 'depth >= %s' % (float(evdep_min)*1000.)
filt2 = 'depth <= %s' % (float(evdep_max)*1000.)
cat = cat.filter(filt1, filt2)

if None not in [evlat_min, evlat_max, evlon_min, evlon_max]:
    filt1 = 'latitude >= %s' % evlat_min
    filt2 = 'latitude <= %s' % evlat_max
    cat = cat.filter(filt1, filt2)

    filt1 = 'longitude >= %s' % evlon_min
    filt2 = 'longitude <= %s' % evlon_max
    cat = cat.filter(filt1, filt2)

#
events=qml_to_event_list(cat)


In [ ]:
ev=events[0] #only work on the first event for now.
print(ev)

### Get station list based on the event information.

In [ ]:
client    = Client('IRIS')
# netall="7D"
stlamin,stlamax,stlomin,stlomax= 37.0,52.0,-133.0,-116.0
Pchanlist=["HDH","BDH"] #pressure channels
# starttime = obspy.UTCDateTime("2012_01_21_0_0_0")       
# endtime   = obspy.UTCDateTime("2012_01_22_0_0_0")
preset=200 #seconds before the origin time
offset=3600 #seconds after the origin time
starttime = ev['datetime'] - preset
endtime = ev['datetime'] + offset

pchanfound=0
try:
    sta_invP = client.get_stations(network='7D',station='*',channel='*DH',location='*', \
                    starttime=starttime,endtime=endtime,minlatitude=stlamin,maxlatitude=stlamax, \
                    minlongitude=stlomin, maxlongitude=stlomax,level='response')

    pchanfound=1
except Exception as e:
    pchanfound == 0

sta,netall,lon,lat,elv,location=sta_info_from_inv(sta_invP)
print(sta,netall,lon,lat,elv,location)          
stalist=sta#["G03A","J35A","J44A","J65A"]



In [ ]:
# get data from IRIS web service
rmresp=True #remove instrument response
rmrespoutput="DISP"
# parameters for butterworth filter
samp_freq=20
pfreqmin=0.005
pfreqmax=samp_freq/2

# prefilter information used when removing instrument responses
f1 = 0.9*pfreqmin;f2=pfreqmin
if 1.1*pfreqmax > 0.45*samp_freq:
    f3 = 0.4*samp_freq
    f4 = 0.45*samp_freq
else:
    f3 = pfreqmax
    f4= 1.1*pfreqmax
pre_filt  = [f1,f2,f3,f4]

# get relationship between water depth and maximum compliance frequency.
d=np.arange(1,5051,50) #water depth
f=maxcompfreq(d,iplot=True)
trmaster=[]
trlabelsmaster=[]
distall=[]
for i in range(len(stalist)):
    ista=stalist[i]
    net=netall[i]
    clear_output(wait=True)
    trall=[] 
    stinv=[]
    trlabels=[]
    """
    array to save all four set of traces: 0-pressure, 1-vertical, 2-prediction of vertical from pressure,
    3-corrected vertical
    """
    
    #download pressure data first
    pchanfound=0
    for chan in Pchanlist:
        try:
            sta_invP = client.get_stations(network=net,station=ista,channel=chan,location='*', \
                            starttime=starttime,endtime=endtime,minlatitude=stlamin,maxlatitude=stlamax, \
                            minlongitude=stlomin, maxlongitude=stlomax,level='response')
            print()
            pchan=chan
            pchanfound=1
            break
        except Exception as e:
            pchanfound == 0
            
    if pchanfound == 0 and chan == Pchanlist[-1]:
        print("Pressure channels not found for "+net+"."+ista)
        continue
    sta,net,lon,lat,elv,location=sta_info_from_inv(sta_invP)
    dist=gps2dist_azimuth(lat[0],lon[0],ev['latitude'],ev['longitude'])
    sta=sta[0]
    net=net[0]
    elv=elv[0]
    stinv.append(sta_invP)
    trlabels.append(net+"."+ista+"."+pchan+":"+str(np.abs(elv))+"m:pressure")
    print("pressure channel "+pchan)
    try:
        tr=client.get_waveforms(network=net,station=ista,\
                        channel=pchan,location="*",starttime=starttime,endtime=endtime)
        tr.detrend("spline", order=3, dspline=500)
        trall.append(tr[0])
    except Exception as e:
        print(e)
        print(trlabels[0])
        continue

    #download vertical data now
    zchan=pchan[0]+"HZ"
    try:
        sta_invZ = client.get_stations(network=net,station=ista,channel=zchan,location='*', \
                        starttime=starttime,endtime=endtime,minlatitude=stlamin,maxlatitude=stlamax, \
                        minlongitude=stlomin, maxlongitude=stlomax,level='response')

        stinv.append(sta_invZ)
    except Exception as e:
        print(e)
        continue
    print("vertical channel "+zchan)
    trlabels.append(net+"."+ista+"."+zchan+":"+str(np.abs(elv))+"m:vertical")
    try:
        tr=client.get_waveforms(network=net,station=ista,\
                        channel=zchan,location="*",starttime=starttime,endtime=endtime)
        tr.detrend("spline", order=3, dspline=500)
        trall.append(tr[0])
    except Exception as e:
        print(e)
        print(trlabels[1])
        continue

    sps=int(trall[0].stats.sampling_rate) #assume pressure and vertical channels have the same sampling rat
    # make downsampling if needed
    if sps > samp_freq:
        # downsampling here
        print("resampling from "+str(sps)+" to "+str(samp_freq))
        for it in range(len(trall)):
            trall[it].interpolate(samp_freq,method='weighted_average_slopes')
            delta = trall[it].stats.delta

            # when starttimes are between sampling points
            fric = trall[it].stats.starttime.microsecond%(delta*1E6)
            if fric>1E-4:
                trall[it].data = segment_interpolate(np.float32(trall[it].data),float(fric/(delta*1E6)))
                #--reset the time to remove the discrepancy---
                trall[it].stats.starttime-=(fric*1E-6)

    #remove response from the two downloaded channels
    if rmresp:
        for it in range(len(trall)):
            if it == 0:continue #skip pressure channel
            if not stinv[it][0][0][0].response:
                raise ValueError('no response found in the inventory! abort!')
            else:
                try:
                    print('removing response for %s using inv'%trall[it])
                    trall[it].attach_response(stinv[it])
                    trall[it].remove_response(output=rmrespoutput,pre_filt=pre_filt,water_level=60)
                except Exception as e:
                    print(e)
                    trall[it] = []
           
    #start compliance correction
    #get the transfer function
    freqmin=pfreqmin
    freqmax=np.around(maxcompfreq(np.abs(elv)),decimals=3)
#     freqmin=0.01
#     freqmax=0.025
    delta=1.0/samp_freq
    print("getting transfer function ...")
#     trall[0].data=1000*trall[0].data #change from kpa to pa
    ff,coh,adm,phs,adm_err,phs_err = gettransfer(trall[0].data,trall[1].data,\
                                                 delta,iplot=False,winlen=250,\
                                                figname=net+"."+ista+"."+pchan+"-"+zchan+"_debug_transfer.png")
    if np.isnan(np.sum(adm)):
        print("getting transfer function found nan in adm. skip to the next station.")
        continue
        
    # do correction now.
    print("doing correction ...")
    trZ_pred,trZ_left = docorrection(trall[0],trall[1],adm,adm_err,phs,phs_err,\
                                     freqmin,freqmax,ff,iplot=0)

    trall.append(trZ_pred)
    trlabels.append(net+"."+ista+"."+zchan+":"+str(np.abs(elv))+"m:prediction")
    trall.append(trZ_left)
    trlabels.append(net+"."+ista+"."+zchan+":"+str(np.abs(elv))+"m:compliance corrected")

    trmaster.append(trall)
    trlabelsmaster.append(trlabels)
    distall.append(dist)
        
    
    
    


In [ ]:
#plotting
print("plotting ...")
pfreqmin=0.02
pfreqmax=0.1
xlimit=[0, 400]
ylimit=[300, 800]
scale=15
figsize=[10,8]
plt.figure(figsize=figsize)
for i in range(len(trmaster)):
    tc=trmaster[i][0].copy()
    tt=tc.times();
    if len(xlimit)>0:
        imin = np.searchsorted(tt,xlimit[0],side="left")
        imax = np.searchsorted(tt,xlimit[1],side="left")
    else:
        imin=0
        imax=len(tc.data)-1
    
    tc.filter('bandpass',freqmin=pfreqmin,freqmax=pfreqmax)
    dtemp=tc.data/np.max(np.abs(tc.data[imin:imax]))
    plt.plot(tt-preset,distall[i][0]/1000 + scale*dtemp,'k')
    plt.xlim(xlimit)
    plt.ylim(ylimit)
    plt.title('Pressure')
# plt.close()

plt.figure(figsize=figsize)
for i in range(len(trmaster)):
    tc=trmaster[i][1].copy()
    tt=tc.times();
    if len(xlimit)>0:
        imin = np.searchsorted(tt,xlimit[0],side="left")
        imax = np.searchsorted(tt,xlimit[1],side="left")
    else:
        imin=0
        imax=len(tc.data)-1
    tc.filter('bandpass',freqmin=pfreqmin,freqmax=pfreqmax)
    dtemp=tc.data/np.max(np.abs(tc.data[imin:imax]))
    plt.plot(tt-preset,distall[i][0]/1000 + scale*dtemp,'b')
    plt.xlim(xlimit)
    plt.ylim(ylimit)
    plt.title('Vertical')
# plt.close()    

plt.figure(figsize=figsize)
for i in range(len(trmaster)):
    tc=trmaster[i][2].copy()
    tt=tc.times();
    if len(xlimit)>0:
        imin = np.searchsorted(tt,xlimit[0],side="left")
        imax = np.searchsorted(tt,xlimit[1],side="left")
    else:
        imin=0
        imax=len(tc.data)-1
    tc.filter('bandpass',freqmin=pfreqmin,freqmax=pfreqmax)
    dtemp=tc.data/np.max(np.abs(tc.data[imin:imax]))
    plt.plot(tt-preset,distall[i][0]/1000 + scale*dtemp,'c')
    plt.xlim(xlimit)
    plt.ylim(ylimit)
    plt.title('Prediction of vertical')
# plt.close()

plt.figure(figsize=figsize)
for i in range(len(trmaster)):
    tc=trmaster[i][3].copy()
    tt=tc.times();
    if len(xlimit)>0:
        imin = np.searchsorted(tt,xlimit[0],side="left")
        imax = np.searchsorted(tt,xlimit[1],side="left")
    else:
        imin=0
        imax=len(tc.data)-1
    tc.filter('bandpass',freqmin=pfreqmin,freqmax=pfreqmax)
    dtemp=tc.data/np.max(np.abs(tc.data[imin:imax]))
    plt.plot(tt-preset,distall[i][0]/1000 + scale*dtemp,'r')
    plt.xlim(xlimit)
    plt.ylim(ylimit)
    plt.title('Vertical after correction')
# plt.close() 